# Task 1

---

## Web scraping and analysis

This Jupyter notebook includes some code to get you started with web scraping. We will use a package called `BeautifulSoup` to collect the data from the web. Once you've collected your data and saved it into a local `.csv` file you should start with your analysis.

### Scraping data from Skytrax

If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.

If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import lazypredict
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time


In [1]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 30 #10
page_size = 100 #100

reviews = []
dates=[]
df = pd.DataFrame()


# for i in range(1, pages + 1):
for i in range(1, pages + 1):
    loop_df = pd.DataFrame()
    count = 0
    table_data = []
    table_headings = []

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("div", {"class": "text_content"}): #text paragraph of review
        reviews.append(para.get_text())

        
    for para10 in parsed_content.find_all('time', itemprop='datePublished'):
        date_text = para10.get_text()
        dates.append(date_text)
        
        
    for para2 in parsed_content.find_all("div", {"class" : "review-stats"}): #table with useful data and ratings
        for para3 in para2.find_all('td',{'class' : 'review-value'}):
            table_data.append(para3.get_text())
#         print(table_data)
        
        rec = table_data[-1] #all reviews have a recommendation - so creating a new variable for this
#         print(rec)
        table_data = table_data[:-1] #not including recommendation as it messes up the formatting - readding it later tho
                
        for para4 in para2.find_all('td',{'class' : 'review-rating-stars stars'}):
            para5 = len(para4.find_all('span', {'class' : 'star fill'}))
            table_data.append(para5)
#         print(table_data)
        
        table_data.append(rec)
        
        for para6 in para2.find_all('td',{'class' : 'review-rating-header'}):
            table_headings.append(para6.get_text())
#         print(table_headings)

        
        dict_data = {table_headings[i]: table_data[i] for i in range(len(table_headings))} #creating a dictionary of the data from table
#         print(dict_data)
        loop_df = pd.DataFrame(dict_data, index=[count])
#         print(loop_df)
        
        df = pd.concat([df,loop_df], ignore_index=True).reset_index(drop=True)
#         print(df)
                        
        table_data = []
        table_headings = []
        count += 1
        
    print(f"   ---> {len(reviews)} total reviews")

df["reviews"] = reviews
df["review date"] = dates

NameError: name 'pd' is not defined

In [13]:
print(df.describe())
df

       Seat Comfort  Cabin Staff Service  Food & Beverages  \
count   2890.000000          2880.000000       2627.000000   
mean       2.812111             3.204167          2.641416   
std        1.360189             1.490266          1.438802   
min        1.000000             1.000000          1.000000   
25%        2.000000             2.000000          1.000000   
50%        3.000000             3.000000          3.000000   
75%        4.000000             5.000000          4.000000   
max        5.000000             5.000000          5.000000   

       Inflight Entertainment  Ground Service  Value For Money  \
count             1891.000000     2795.000000      3000.000000   
mean                 2.662612        2.800000         2.581333   
std                  1.390966        1.455275         1.451007   
min                  1.000000        1.000000         1.000000   
25%                  1.000000        1.000000         1.000000   
50%                  3.000000        3.000000

,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Ground Service,Value For Money,Recommended,Wifi & Connectivity,reviews,review date
0,A380,Solo Leisure,Business Class,London to Miami,September 2023,1.0,1.0,2.0,1.0,3.0,1,no,NaN,✅ Trip Verified | Flying A380 business class ...,2nd September 2023
1,Boeing 777,NaN,Economy Class,Los Angeles to London,August 2023,1.0,1.0,1.0,1.0,1.0,1,no,NaN,✅ Trip Verified | British Airways absolutely ...,1st September 2023
2,NaN,Couple Leisure,Economy Class,New York JFK to Malaga via London,June 2023,1.0,3.0,NaN,NaN,1.0,1,no,NaN,✅ Trip Verified | My recent experience with B...,1st September 2023
3,NaN,Couple Leisure,Premium Economy,Chicago to Rome via London,July 2023,3.0,1.0,1.0,1.0,1.0,1,no,1.0,✅ Trip Verified | This is to express our disp...,31st August 2023
4,A320,Solo Leisure,Business Class,London to Malaga,August 2023,1.0,1.0,1.0,1.0,1.0,1,no,1.0,✅ Trip Verified | I flew London to Malaga on ...,30th August 2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,NaN,NaN,Business Class,NaN,NaN,1.0,5.0,3.0,4.0,NaN,1,no,NaN,I travelled from London to Jo'burg and back on...,8th April 2015
2996,NaN,NaN,Economy Class,NaN,NaN,5.0,4.0,4.0,4.0,NaN,5,yes,NaN,From Shanghai to London on 17 Mar. The whole t...,8th April 2015
2997,NaN,NaN,Economy Class,NaN,NaN,2.0,5.0,4.0,1.0,NaN,4,yes,NaN,Flew Warsaw to Heathrow. Reasonable all round....,8th April 2015
2998,NaN,NaN,Economy Class,NaN,NaN,3.0,2.0,3.0,1.0,NaN,4,no,NaN,Return LHR-YVR Boeing 747-400. I had been advi...,6th April 2015


In [14]:
data = df.head(100)
print(data.describe())

       Seat Comfort  Cabin Staff Service  Food & Beverages  \
count     92.000000            92.000000         73.000000   
mean       2.347826             2.750000          2.260274   
std        1.362244             1.537981          1.343993   
min        1.000000             1.000000          1.000000   
25%        1.000000             1.000000          1.000000   
50%        2.000000             3.000000          2.000000   
75%        3.000000             4.000000          3.000000   
max        5.000000             5.000000          5.000000   

       Inflight Entertainment  Ground Service  Value For Money  \
count               51.000000       95.000000       100.000000   
mean                 2.431373        1.926316         1.760000   
std                  1.513339        1.416034         1.223672   
min                  1.000000        1.000000         1.000000   
25%                  1.000000        1.000000         1.000000   
50%                  2.000000        1.000000

In [15]:
df2 = df.head(1000)
print(df2.describe())

       Seat Comfort  Cabin Staff Service  Food & Beverages  \
count    915.000000           909.000000        789.000000   
mean       2.784699             3.195820          2.702155   
std        1.391041             1.527697          1.481273   
min        1.000000             1.000000          1.000000   
25%        1.000000             2.000000          1.000000   
50%        3.000000             3.000000          3.000000   
75%        4.000000             5.000000          4.000000   
max        5.000000             5.000000          5.000000   

       Inflight Entertainment  Ground Service  Value For Money  \
count              543.000000      943.000000      1000.000000   
mean                 2.795580        2.660657         2.430000   
std                  1.471945        1.543912         1.489066   
min                  1.000000        1.000000         1.000000   
25%                  1.000000        1.000000         1.000000   
50%                  3.000000        3.000000

In [16]:
df.to_csv("data/BA_reviews - 3000 reviews.csv")
# df1.to_csv("data/BA_reviews - 2000 reviews.csv")
df2.to_csv("data/BA_reviews - 1000 reviews.csv")
data.to_csv("data/BA_reviews - 100 reviews.csv")
print('save complete')


save complete


Congratulations! Now you have your dataset for this task! The loops above collected 1000 reviews by iterating through the paginated pages on the website. However, if you want to collect more data, try increasing the number of pages!

 The next thing that you should do is clean this data to remove any unnecessary text from each of the rows. For example, "✅ Trip Verified" can be removed from each row if it exists, as it's not relevant to what we want to investigate.

In [17]:
'''
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 1 #10
page_size = 100 #100

reviews = []
# aircraft = []
# seat_type = []
# route = []
# recommended = []
df = pd.DataFrame()

temp_df = pd.DataFrame()
rdm_df = pd.DataFrame()


# for i in range(1, pages + 1):
for i in range(1, pages + 1):
    count = 0
    table_data = []
    table_headings = []

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("div", {"class": "text_content"}): #text paragraph of review
        reviews.append(para.get_text())
        
    for para2 in parsed_content.find_all("div", {"class" : "review-stats"}): #table with useful data and ratings
        for para3 in para2.find_all('td',{'class' : 'review-value'}):
            table_data.append(para3.get_text())
        print(table_data)
        
        rec = table_data[-1] #all reviews have a recomendation - so creating a new variable for this
        print(rec)
        
            
#         recomend = rating[-1]
#         print(recomend)
#         rating = rating[:-1]
#         print(rating)
        
        for para4 in para2.find_all('td',{'class' : 'review-rating-stars stars'}):
            para5 = len(para4.find_all('span', {'class' : 'star fill'}))
            table_data.append(para5)
#         table_data.append(rec)
        print(table_data)
        
        for para6 in para2.find_all('td',{'class' : 'review-rating-header'}):
            table_headings.append(para6.get_text())
        print(table_headings)
        
        
        dict_data = {table_headings[i]: table_data[i] for i in range(len(table_headings))} #creating a dictionary of the data from table
        print(dict_data)
        temp_df = pd.DataFrame(dict_data, index=[count])
        print(temp_df)
        
        rdm_df = pd.concat([rdm_df,temp_df], ignore_index=True).reset_index(drop=True)
        print(rdm_df)
            
            
            
            
        #print(category)
        # Create the records from both list, using zip and dict calls.

        # Build the dataframe from the dictionary.
#         res = {}
#             for key in test_keys:
#                 for value in test_values:
#                     res[key] = value
#                     test_values.remove(value)
#                     break
#         temp_df = pd.DataFrame(table_data, columns = table_headings, axis=0)



#         res = {table_headings[i]: table_data[i] for i in range(len(table_headings))}
    
#         for key, value in res.items():
#             print(key)
#             temp_df[key]
#             print(temp_df)

#             headings = key
            
#             values = value
#             print(value)
#             temp_df[key] = value
            
#         temp_df = pd.DataFrame(res.items())
#         print(temp_df)

    
    
    
        
#         data_dict = pd.DataFrame([table_data], columns=table_headings)
#         print(data_dict.head(2))
#         df = df.append(data_dict, ignore_index=True).reset_index(drop=True)
#         df = pd.concat(res, ignore_index=True, axis=1).reset_index(drop=True)

        #print(df)
        table_data = []
        table_headings = []
        count += 1
        
    print(f"   ---> {len(reviews)} total reviews")

df["reviews"] = reviews
'''

'\nbase_url = "https://www.airlinequality.com/airline-reviews/british-airways"\npages = 1 #10\npage_size = 100 #100\n\nreviews = []\n# aircraft = []\n# seat_type = []\n# route = []\n# recommended = []\ndf = pd.DataFrame()\n\ntemp_df = pd.DataFrame()\nrdm_df = pd.DataFrame()\n\n\n# for i in range(1, pages + 1):\nfor i in range(1, pages + 1):\n    count = 0\n    table_data = []\n    table_headings = []\n\n    print(f"Scraping page {i}")\n\n    # Create URL to collect links from paginated data\n    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"\n\n    # Collect HTML data from this page\n    response = requests.get(url)\n\n    # Parse content\n    content = response.content\n    parsed_content = BeautifulSoup(content, \'html.parser\')\n    for para in parsed_content.find_all("div", {"class": "text_content"}): #text paragraph of review\n        reviews.append(para.get_text())\n        \n    for para2 in parsed_content.find_all("div", {"class" : "review-stats"}): 